In [65]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [66]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [67]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

In [74]:
import pymongo
from pymongo import MongoClient
import json
from pprint import pprint
from tqdm.auto import tqdm
import re

openai_api_key  = get_secret('openai')['api_key']

mongodb_connect = get_secret('mongodb1')['api_key']

In [97]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
job_data_collection = db.job_data # make sure the collected jobs unique

In [98]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)
model="gpt-4o"
temperature=0

def openai_help(prompt, model=model, temperature =temperature ):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

In [99]:
filter={

    
}
project={
    'QualificationSummary': 1, 
    'PositionID': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['job_data'].find(
  filter=filter,
  projection=project
)

In [100]:
job_data = []
for job in result:
    job_data.append({'Position_id':job['PositionID'],
                     'summary':job['QualificationSummary']})

In [101]:
print('Number of jobs: ',len(job_data))

Number of jobs:  955


In [ ]:
for job in tqdm(job_data):
  
    prompt = f"""
    Identify the common technology skills from the job summary, 
    job summary: {job['summary']},
    format the items in a JSON  list,
    if no technlology skills presented, use "Unknown" in the list.
    Do not wrap the JSON codes in JSON markers

 
    """
#     print(prompt)

    extract_result =openai_help(prompt)
    print(extract_result)

    job_data_collection.update_one(
            {'PositionID':job['Position_id']},
            {"$set":{'skills':json.loads(extract_result)}}
        )


  0%|          | 0/955 [00:00<?, ?it/s]

[
    "Unknown"
]
[
    "Unknown"
]
[
    "Microsoft programs and software"
]
[
    "Unknown"
]
[
    "Information Management",
    "Information Technology Policy and Planning",
    "Project Management"
]
[
    "Unknown"
]
[
    "Unknown"
]
[
    "Information Technology (IT) related experience",
    "Attention to Detail",
    "Customer Service",
    "Oral Communication",
    "Problem Solving",
    "Interviewing subject-matter personnel",
    "Synthesizing data into charts",
    "Operating computer consoles",
    "Scheduling programs for computers",
    "Preparing documentation on cost/benefit studies",
    "Working with customers to establish or change accounts",
    "Translating logical steps into programming language codes",
    "Analysis of system components",
    "Planning sequence of actions for assignments",
    "Personal responsibility for project segments",
    "Developing modifications to system parts",
    "Knowledge of computer applications or specialty areas",
    "Planning